## QAOA for edge coloring problem

We start by importing packages that will be utilized throughout the subsequent programs.

In [ ]:
import matplotlib.pyplot as plt
import random
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.quantum_info.states.quantum_state import QuantumState
import qiskit.quantum_info as qi
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
import numpy as np
from numpy import pi
from scipy.optimize import minimize
from operator import itemgetter
import networkx as nx
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
import seaborn as sns


First, we input the graph as a set of nodes and edges and create a visual representation of it by drawing a sketch using NetworkX.

In [ ]:
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3, 4, 5])
G.add_edges_from([(3, 1), (3, 2), (0, 3), (0, 4), (1, 5), (1, 4), (3, 4), (2, 4)])
nx.draw(G, with_labels=True, alpha=0.8, node_size=500)

Next, we define two quantum circuits that implement the two mixer Hamiltonians with a parameter beta.

In [ ]:
V = G.number_of_nodes()
E = G.number_of_edges()    
    
def QC_mixOLD(beta):
    QC_mixBeta = QuantumCircuit(2 * E, 2 * E)
    for i in range(2 * E):
        QC_mixBeta.rx(2 * beta, i)
    return QC_mixBeta 
    
def QC_mixNEW(beta):
    QC_mixBeta = QuantumCircuit(2 * E, 2 * E)
    for i in range(2 * E):
        QC_mixBeta.h(i)
    for i in range(E):
        QC_mixBeta.x(2*i)
        QC_mixBeta.x(2*i+1)
        QC_mixBeta.cp(-4*beta, 2*i, 2*i+1)
        QC_mixBeta.x(2*i)
        QC_mixBeta.x(2*i+1)
    for i in range(2 * E):
        QC_mixBeta.h(i)
    return QC_mixBeta

We proceed by defining a quantum circuit that implements the problem Hamiltonian  with a parameter gamma for edge coloring problem.

In [ ]:
EdgeNumber = {}        
  
curNumber = 0    
for i in G.edges():
    EdgeNumber[i] = curNumber
    print('edge ' + str(i) + ' has number ' + str(curNumber))
    curNumber += 1

AdjacentEdges = {}    

A = nx.to_pandas_adjacency(G, dtype=int)

for z in G.edges():
    i = z[0]
    j = z[1]
    l = EdgeNumber[(i,j)]
    AdjacentEdges[l] = []

for z in G.edges():
    i = z[0]
    j = z[1]
    l = EdgeNumber[(i,j)]
    for k in range(0, V):
        if A[i][k] != 0 and k != j:
            if (i,k) in EdgeNumber:
                t = EdgeNumber[(i,k)]
            else:
                t = EdgeNumber[(k,i)]
            if l not in AdjacentEdges[t]:
                AdjacentEdges[l].append(t)
    for k in range(0, V):
        if A[j][k] != 0 and k != i:
            if (j,k) in EdgeNumber:
                t = EdgeNumber[(j,k)]
            else:
                t = EdgeNumber[(k,j)]
            if l not in AdjacentEdges[t]:
                AdjacentEdges[l].append(t)

for i in AdjacentEdges:
    print('edge number ' + str(i) + ' shares a vertex with edges ' + str(AdjacentEdges[i]))        

def QC_problem(gamma):
    QC_problemGamma = QuantumCircuit(2 * E, 2 * E)
    for i in range(0, E):
        for j in AdjacentEdges[i]:
            QC_problemGamma.cx(2*i,2*i+1)
            QC_problemGamma.cx(2*i+1,2*j)
            QC_problemGamma.cx(2*j,2*j+1)
            QC_problemGamma.rz(2 * gamma, 2*j+1)
            QC_problemGamma.cx(2*j,2*j+1)
            QC_problemGamma.cx(2*i+1,2*j)
            QC_problemGamma.cx(2*i,2*i+1)
        
            QC_problemGamma.cx(2*i,2*j)
            QC_problemGamma.rz(2 * gamma, 2*j)
            QC_problemGamma.cx(2*i,2*j)
        
            QC_problemGamma.cx(2*i+1,2*j+1)
            QC_problemGamma.rz(2 * gamma, 2*j+1)
            QC_problemGamma.cx(2*i+1,2*j+1)
        
            QC_problemGamma.barrier()
            
    return QC_problemGamma

We conclude this section by presenting the quantum circuits for two variations of QAOA, which are determined by the choice of mixer Hamiltonian. Each circuit takes an input depth parameter p, as well as two arrays of size p for the beta and gamma angle parameters.

In [ ]:
def QAOAold(beta, gamma, p):
    QAOAcircuit = QuantumCircuit(2 * E, 2 * E)
    for i in range(0, 2 * E):
        QAOAcircuit.h(i)
    for i in range(p):
        QAOAcircuit = QAOAcircuit.compose(QC_problem(gamma[i])) 
        QAOAcircuit = QAOAcircuit.compose(QC_mixOLD(beta[i])) 
    QAOAcircuit.measure(range(2*E), range(2*E))
    return QAOAcircuit

def QAOAnew(beta, gamma, p):
    QAOAcircuit = QuantumCircuit(2 * E, 2 * E)
    for i in range(0, 2 * E):
        QAOAcircuit.h(i)
    for i in range(p):
        QAOAcircuit = QAOAcircuit.compose(QC_problem(gamma[i])) 
        QAOAcircuit = QAOAcircuit.compose(QC_mixNEW(beta[i])) 
    QAOAcircuit.measure(range(2*E), range(2*E))
    return QAOAcircuit

We define the objective function for the edge coloring problem and the energy function as the average value of the objective function over the samples under consideration.

In [ ]:
def invert_counts(counts):
    return {k[::-1]:v for k, v in counts.items()}

def coloring_obj(x):
    val = 0
    for i in range(0, E):
        for j in AdjacentEdges[i]:
            if x[2*i] == x[2*j] and x[2*i+1] == x[2*j+1]:
                # pair of adjacent edges of coinciding color
                val += 1
    return val

def compute_coloring_energy(counts):
    energy = 0
    total_counts = 0
    for meas, meas_count in counts.items():
        obj_for_meas = coloring_obj(meas)
        energy += obj_for_meas * meas_count
        total_counts += meas_count
    return energy / total_counts

backend = Aer.get_backend('qasm_simulator')

def get_black_box_objectiveOLD(p):
    def f(theta):
        # let's assume first half is betas, second half is gammas
        beta = theta[:p]
        gamma = theta[p:]
        qc = QAOAold(beta, gamma, p)
        counts = execute(qc, backend, seed_simulator=10).result().get_counts()
        # return the energy
        return compute_coloring_energy(invert_counts(counts))
    return f

def get_black_box_objectiveNEW(p):
    def f(theta):
        # let's assume first half is betas, second half is gammas
        beta = theta[:p]
        gamma = theta[p:]
        qc = QAOAnew(beta, gamma, p)
        counts = execute(qc, backend, seed_simulator=10).result().get_counts()
        # return the energy
        return compute_coloring_energy(invert_counts(counts))
    return f

We execute both algorithms iteratively with a depth parameter set to p=9. Subsequently, we store the resulting energy values in arrays named NineValG_old and NineValG_new, respectively.

In [ ]:
NineValG_old=[]

for t in range(50):
    print('iteration=' + str(t+1))
    
    objective_valuesOLD = []
    Pvalue = []
    
    old_param = np.array([])
    
    arr_old = np.array([np.pi*random.random(), 2*np.pi*random.random()])
    print('Starting parameters old ' + str(arr_old))

    for p in range(1, 10):
    
        init_point_old = arr_old
        objOLD = get_black_box_objectiveOLD(p)
        resOLD = minimize(objOLD, init_point_old, method='COBYLA', options={'maxiter': 500, 'disp': True})
    
        # Append the objective value to the list
        Pvalue.append(p)
        objective_valuesOLD.append(resOLD['fun'])
        
        arr_old = np.array([])
        temp = resOLD['x']
        
        for i in range(p):
            arr_old = np.append(arr_old, temp[i])
        arr_old = np.append(arr_old, 0.0)
        for i in range(p):
            arr_old = np.append(arr_old, temp[p+i])
        arr_old = np.append(arr_old, 0.0)
    
        
    NineValG_old.append(resOLD['fun'])
print(NineValG_old)

for t in range(50):
    print('iteration=' + str(t+1))
    
    objective_valuesNEW = []
    Pvalue = []
    
    new_param = np.array([])
    
    arr_new = np.array([0.25*np.pi*random.random(), 2*np.pi*random.random()])
    print('Starting parameters new ' + str(arr_new))

    for p in range(1, 10):
    
        init_point_new = arr_new
        objNEW = get_black_box_objectiveNEW(p)
        resNEW = minimize(objNEW, init_point_new, method='COBYLA', options={'maxiter': 500, 'disp': True})
    
        # Append the objective value to the list
        Pvalue.append(p)
        objective_valuesNEW.append(resNEW['fun'])
        
    
        arr_new = np.array([])
        temp = resNEW['x']
    
        for i in range(p):
            arr_new = np.append(arr_new, temp[i])
        arr_new = np.append(arr_new, 0.0)
        for i in range(p):
            arr_new = np.append(arr_new, temp[p+i])
        arr_new = np.append(arr_new, 0.0)
        
    NineValG_new.append(resNEW['fun'])
print(NineValG_new)

We finish by collecting the statistical data.

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
import seaborn as sns
import matplotlib.pyplot as plt


# Visual Inspection
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(NineValG_old, kde=True, color='blue', label='val_old')
sns.histplot(NineValG_new, kde=True, color='orange', label='val_new')
plt.legend()

plt.subplot(1, 2, 2)
stats.probplot(NineValG_old, dist='norm', plot=plt)
stats.probplot(NineValG_new, dist='norm', plot=plt)
plt.show()

# Descriptive Statistics
print("val_old Mean:", np.mean(NineValG_old))
print("val_new Mean:", np.mean(NineValG_new))
print("val_old Median:", np.median(NineValG_old))
print("val_new Median:", np.median(NineValG_new))
print("val_old Skewness:", stats.skew(NineValG_old))
print("val_new Skewness:", stats.skew(NineValG_new))
print("val_old Kurtosis:", stats.kurtosis(NineValG_old))
print("val_new Kurtosis:", stats.kurtosis(NineValG_new))

# Shapiro-Wilk Test
shapiro_stat_1, shapiro_p_value_old = stats.shapiro(NineValG_old)
shapiro_stat_2, shapiro_p_value_new = stats.shapiro(NineValG_new)
print("Shapiro-Wilk Test - val_old:", shapiro_p_value_old)
print("Shapiro-Wilk Test - val_new:", shapiro_p_value_new)

# Anderson-Darling Test
anderson_stat_1 = stats.anderson(NineValG_old)
anderson_stat_2 = stats.anderson(NineValG_new)
print("Anderson-Darling Test - val_old:", anderson_stat_1)
print("Anderson-Darling Test - val_new:", anderson_stat_2)

# Kernel Density Plot
plt.figure(figsize=(12, 6))
sns.kdeplot(NineValG_old, color='blue', label='val_old', shade=True)
sns.kdeplot(NineValG_new, color='orange', label='val_new', shade=True)
plt.legend()
plt.show()

t_stat, p_value = ttest_ind(NineValG_old, NineValG_new)
w_stat, wp_value = mannwhitneyu(NineValG_old, NineValG_new)
print('t-test p-value is '+ str(p_value))
print('wp-test p-value is '+ str(wp_value))

## QAOA for graph partioning problem

The cell below presents analogous Python code for solving the graph partitioning problem on a cyclic graph with 4 vertices.

In [ ]:
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])


V = G.number_of_nodes()
E = G.number_of_edges()

punishment = E

Nodes = []
AdjacentNodes = {}       

for i in range(V):
    Nodes.append(i)
    AdjacentNodes[i] = []

for z in G.edges():
    i = z[0]
    j = z[1]
    if j not in AdjacentNodes[i]:
        AdjacentNodes[i].append(j)  
        
def QC_mixOLD(beta):
    QC_mixBeta = QuantumCircuit(2 * V + 1, 2 * V + 1)
    for i in range(2 * V):
        QC_mixBeta.h(i)
    for i in range(V):
    
def QC_mixNEW(beta):
    QC_mixBeta = QuantumCircuit(2 * V, 2 * V)
    for i in range(2 * V):
        QC_mixBeta.h(i)
    for i in range(V):
        QC_mixBeta.x(2*i)
        QC_mixBeta.x(2*i+1)
        QC_mixBeta.cp(-4*beta, 2*i, 2*i+1)
        QC_mixBeta.x(2*i)
        QC_mixBeta.x(2*i+1)
    for i in range(2 * V):
        QC_mixBeta.h(i)
    return QC_mixBeta

def QC_problem(gamma):
    QC_problemGamma = QuantumCircuit(2 * V, 2 * V)
      
    for i in range(0, V):
        for j in AdjacentNodes[i]:
            QC_problemGamma.cx(2*i,2*i+1)
            QC_problemGamma.cx(2*i+1,2*j)
            QC_problemGamma.cx(2*j,2*j+1)
            QC_problemGamma.rz(-2 * gamma, 2*j+1)
            QC_problemGamma.cx(2*j,2*j+1)
            QC_problemGamma.cx(2*i+1,2*j)
            QC_problemGamma.cx(2*i,2*i+1)
        
            QC_problemGamma.cx(2*i,2*j)
            QC_problemGamma.rz(-2 * gamma, 2*j)
            QC_problemGamma.cx(2*i,2*j)
        
            QC_problemGamma.cx(2*i+1,2*j+1)
            QC_problemGamma.rz(-2 * gamma, 2*j+1)
            QC_problemGamma.cx(2*i+1,2*j+1)
    
    for i in range(0, V-1): 
        for j in range(i+1, V): 
            
            QC_problemGamma.cx(2*i, 2*j)
            QC_problemGamma.rz(2 * punishment * gamma, 2*j)
            QC_problemGamma.cx(2*i, 2*j)    
        
    for i in range(0, V-1): 
        for j in range(i+1, V): 
            
            QC_problemGamma.cx(2*i,2*i+1)
            QC_problemGamma.cx(2*i+1,2*j)
            QC_problemGamma.cx(2*j,2*j+1)
            QC_problemGamma.rz(4 * punishment * gamma, 2*j+1)
            QC_problemGamma.cx(2*j,2*j+1)
            QC_problemGamma.cx(2*i+1,2*j)
            QC_problemGamma.cx(2*i,2*i+1)
            
            QC_problemGamma.cx(2*i+1,2*j+1)
            QC_problemGamma.rz(4 * punishment * gamma, 2*j+1)
            QC_problemGamma.cx(2*i+1,2*j+1)        
        
    return QC_problemGamma

def QAOAold(beta, gamma, p):
    QAOAcircuit = QuantumCircuit(2 * V, 2 * V)
    for i in range(0, 2 * V):
        QAOAcircuit.h(i)
    for i in range(p):
        QAOAcircuit = QAOAcircuit.compose(QC_problem(gamma[i])) 
        QAOAcircuit = QAOAcircuit.compose(QC_mixOLD(beta[i])) 
    QAOAcircuit.measure(range(2*V), range(2*V))
    return QAOAcircuit

def QAOAnew(beta, gamma, p):
    QAOAcircuit = QuantumCircuit(2 * V, 2 * V)
    for i in range(0, 2 * V):
        QAOAcircuit.h(i)
    for i in range(p):
        QAOAcircuit = QAOAcircuit.compose(QC_problem(gamma[i])) 
        QAOAcircuit = QAOAcircuit.compose(QC_mixNEW(beta[i])) 
    QAOAcircuit.measure(range(2*V), range(2*V))
    return QAOAcircuit

def invert_counts(counts):
    return {k[::-1]:v for k, v in counts.items()}

def coloring_obj(x):
    val = 0
    for i in range(0, V):
        for j in AdjacentNodes[i]:
            if x[2*i] != x[2*j] or x[2*i+1] != x[2*j+1]:
            # pair of adjacent nodes of different color
                val += 1
    
    err = 0
    
    for i in range(0, V):
        err += punishment * 2 * (int(x[2*i])-0.5)
    
    val += abs(err)
        
    err = 0
    
    for i in range(0, V):
        err += punishment * 2 * (int(x[2*i+1])-0.5) * (1-int(x[2*i]))
                
    val += abs(err)
        
    err = 0  
    
    for i in range(0, V):
        err += punishment * 2 * (int(x[2*i+1])-0.5) * int(x[2*i])
                
    val += abs(err)
            
    return val

def compute_coloring_energy(counts):
    energy = 0
    total_counts = 0
    minEnergycounts = 0
    for meas, meas_count in counts.items():
        obj_for_meas = coloring_obj(meas)
        energy += obj_for_meas * meas_count
        total_counts += meas_count
        if (obj_for_meas == 4):
            minEnergycounts += meas_count
    
    print('success probability is '+ str(100*minEnergycounts/total_counts)+'%')
    return energy / total_counts



def get_black_box_objectiveOLD(p):
    backend = Aer.get_backend('qasm_simulator')
    def f(theta):
        # let's assume first half is betas, second half is gammas
        beta = theta[:p]
        gamma = theta[p:]
        qc = QAOAold(beta, gamma, p)
        counts = execute(qc, backend, seed_simulator=10).result().get_counts()
        # return the energy
        return compute_coloring_energy(invert_counts(counts))
    return f

def get_black_box_objectiveNEW(p):
    backend = Aer.get_backend('qasm_simulator')
    def f(theta):
        # let's assume first half is betas, second half is gammas
        beta = theta[:p]
        gamma = theta[p:]
        qc = QAOAnew(beta, gamma, p)
        counts = execute(qc, backend, seed_simulator=10).result().get_counts()
        # return the energy
        return compute_coloring_energy(invert_counts(counts))
    return f

NineValGG_new=[]
NineValGG_old=[]

for t in range(50):
    print('iteration=' + str(t+1))
    
    objective_valuesNEW = []
    Pvalue = []
    
    old_param = np.array([])
    
    arr_old = np.array([0.25*np.pi*random.random(), 0.2*np.pi*random.random()])
    print('Starting parameters new ' + str(arr_old))

    for p in range(1, 8):
    
        init_point_old = arr_old
        objOLD = get_black_box_objectiveOLD(p)
        resOLD = minimize(objOLD, init_point_old, method='COBYLA', options={'maxiter': 200, 'disp': True})
        
    
        # Append the objective value to the list
        Pvalue.append(p)
        objective_valuesOLD.append(resOLD['fun'])
        
    
        arr_old = np.array([])
        temp = resOLD['x']
    
        for i in range(p):
            arr_old = np.append(arr_old, temp[i])
        arr_old = np.append(arr_old, 0.0)
        for i in range(p):
            arr_old = np.append(arr_old, temp[p+i])
        arr_old = np.append(arr_old, 0.0)
          
    NineValGG_old.append(resOLD['fun'])
print(NineValGG_old)

for t in range(50):
    print('iteration=' + str(t+1))
    
    objective_valuesNEW = []
    Pvalue = []
    
    new_param = np.array([])
    
    arr_new = np.array([0.25*np.pi*random.random(), 0.2*np.pi*random.random()])
    print('Starting parameters new ' + str(arr_new))

    for p in range(1, 8):
    
        init_point_new = arr_new
        objNEW = get_black_box_objectiveNEW(p)
        resNEW = minimize(objNEW, init_point_new, method='COBYLA', options={'maxiter': 200, 'disp': True})
        
    
        # Append the objective value to the list
        Pvalue.append(p)
        objective_valuesNEW.append(resNEW['fun'])
        
    
        arr_new = np.array([])
        temp = resNEW['x']
    
        for i in range(p):
            arr_new = np.append(arr_new, temp[i])
        arr_new = np.append(arr_new, 0.0)
        for i in range(p):
            arr_new = np.append(arr_new, temp[p+i])
        arr_new = np.append(arr_new, 0.0)
          
    NineValGG_new.append(resNEW['fun'])
print(NineValGG_new)